In [1]:
using Plots;plotly()
using Optim
using Zygote
using Meteor.Utility
using Meteor.ExactDiag
using DataFrames;using CSV
using Pkg

In [2]:
using LinearAlgebra
ccall((:openblas_get_num_threads64_, Base.libblas_name),Cint,())
LinearAlgebra.BLAS.set_num_threads(8)

In [3]:
using Base.Threads;nthreads()

1

In [4]:
function thermal_state(dims,nbar)
    c::Array{Complex{Float64}}=zeros(dims,dims)
    if nbar==0
        c[1,1]=1
    else
        for i in 1:dims
           c[i,i]= exp(-i*log(1/nbar+1))
        end
    end
    c=c/tr(c)
    return c        
end

function fock_state(dims,n)
    c::Matrix{ComplexF64}=zeros(dims,dims)
    if n<dims
        c[n+1,n+1]=1
    end
    return c
end

function eye(n)
    k::Array{Complex{Float64}}=zeros(n,n)
    for i in 1:n
        k[i,i]=1
    end
    return k    
end


function a₋(n)      #n为维数
    s::Array{Complex{Float64}}=zeros(n,n)
    for i=1:n-1
        s[i,i+1]=sqrt(i)
    end
    return s
end

a₊(n)=adjoint(a₋(n));

dim=10
ν=1
dot_num=30
initial_state=kron([1.0 0 0;0 0 0;0 0 0],thermal_state(dim,1))
#########密度矩阵向量化，超算子维数变成n^2*n^2维,态矩阵厄米，只用存一半？意义不大
function dm2vec(dm)#态向量化
#julia里是按列reshape的,reshape成列向量
    return reshape(dm,:,1)
end

function vec2dm(vec)
    return reshape(vec,3*dim,:)
end


n=kron(eye(3),a₊(dim)*a₋(dim))
en=kron([0 0 0;0 0 0;0 0 1],eye(dim))
rr=kron([0 0 0;0 1 0;0 0 0],eye(dim))
η1=0.15
η2=-0.15
a=a₊(dim)+a₋(dim)
ag2=kron([0 0 0;0 0 0;1 0 0],exp(im*η1*a))
ag3=kron([0 0 1;0 0 0;0 0 0],exp(-im*η1*a))
ag4=ag2+ag3
ar2=kron([0 0 0;0 0 0;0 1 0],exp(im*η2*a))
ar3=kron([0 0 0;0 0 1;0 0 0],exp(-im*η2*a))
ar4=ar2+ar3

E=eye(3*dim)
γ1=40/3*ν
γ2=20/3*ν

function diss1()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η1*theta*a)
        aup=kron([0 0 0;0 0 0;1 0 0],jump')
        adown=kron([0 0 1;0 0 0;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end
function diss2()
    dtheta = 0.01
    x=zeros((3*dim)^2,(3*dim)^2)
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*η2*theta*a)
        aup=kron([0 0 0;0 0 0;0 1 0],jump')
        adown=kron([0 0 0;0 0 1;0 0 0],jump)
        x=x+kron(transpose(aup),adown)-0.5*kron(E,aup*adown)-0.5*kron(transpose(aup*adown),E)
    end
    return x
end

dissipation1=γ1*diss1()
dissipation2=γ2*diss2()

# Δg=(Ωr^2+Ωg^2)/ν-ν
# Δr=Δg

function eit_Lindbladian_vec(Ωg,Ωr,Δ)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δ*en+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);
ttot=300
function feval(x)
    f=exp(ttot*eit_Lindbladian_vec(x[1],x[2],x[3]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end

function g!(G,seq)
    print("g!  ")
    a=real(gradient(feval,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end

g! (generic function with 1 method)

In [18]:
# 先无梯度搜索，这里用粒子群算法，防止跳入局部最优解，并且这样的计算速度也可以比较快的从离正确值较远的地方找到一个差不多的解
# @time res=optimize(feval,[0,0,-500],[100,100,500],[3.6103223565556357,21.05702368447422,111.9645147569779],ParticleSwarm(),Optim.Options(iterations=30))

In [19]:
@time res=optimize(feval,g!,[3.6103223565556357,21.05702368447422,111.9645147569779],LBFGS(),Optim.Options(x_tol=1e-4))

g!  0.0067549313150255105 0.00675493131502651   4.715945 seconds (477.30 k allocations: 1.290 GiB, 9.09% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     6.754931e-03

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 1.0e-04
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = 1.01e-09 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1


In [20]:
Optim.minimum(res)

0.00675493131502651

In [21]:
se1=Optim.minimizer(res)

3-element Array{Float64,1}:
   3.6103223565556357
  21.05702368447422
 111.9645147569779

In [23]:
function eit_Lindbladian_vec2(Ωg,Ωr,Δg,Δr)
    unitary::Matrix{ComplexF64}=kron(E,-im*(ν*n-Δg*en-(Δg-Δr)rr+Ωg/2*ag4+Ωr/2*ar4))+kron(transpose(im*((ν*n-Δg*en-(Δg-Δr)rr+Ωg/2*ag4+Ωr/2*ar4))),E)
    
    return unitary+dissipation1+dissipation2
end

instate=dm2vec(initial_state);
ttot=100
function feval2(x)
    f=exp(ttot*eit_Lindbladian_vec2(x[1],x[2],x[3],x[4]))*instate
    a=real(tr(n*vec2dm(f)))
    print(a," ")
    return a
end

function g2!(G,seq)
    print("g!  ")
    a=real(gradient(feval2,seq)[1])
    for i in 1:length(seq)
       G[i]=a[i] 
    end
end

g2! (generic function with 1 method)

In [27]:
@time res2=optimize(feval2,g2!,[5.281092614609339, 16.28498178068964,58.908208592426824,58.682468257711506],LBFGS(),Optim.Options(x_tol=1e-4))

g!  0.03708405068770156 0.03708405068769825   3.672562 seconds (2.08 k allocations: 1.196 GiB, 9.96% gc time)


 * Status: success

 * Candidate solution
    Final objective value:     3.708405e-02

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 0.00e+00 ≤ 1.0e-04
    |x - x'|/|x'|          = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|         = NaN ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = NaN ≰ 0.0e+00
    |g(x)|                 = 4.47e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    0
    f(x) calls:    1
    ∇f(x) calls:   1


In [28]:
Optim.minimum(res2)

0.03708405068769825

In [26]:
se2=Optim.minimizer(res2)

4-element Array{Float64,1}:
  5.281092614609339
 16.28498178068964
 58.908208592426824
 58.682468257711506

In [6]:
function eit_cooling(d, Delta, nu, eta_g, omega_g, eta_r, omega_r, gamma_g, gamma_r, state, t)
    ps = nlevel(["g", "r", "e"])
    pb = boson(d=d)
    
    xop = pb["a"] + pb["adag"]
    
    ham = Lindbladian([ps, pb])
    add_unitary!(ham, (1,), ("e<-e",), coeff=-Delta)
    add_unitary!(ham, (2,), ("n",), coeff=nu)
    
    exp_xop = exp(im*eta_g*xop)
    add_unitary!(ham, (1, 2), ("e<-g", exp_xop), coeff=omega_g/2)
    add_unitary!(ham, (1, 2), ("g<-e", exp_xop'), coeff=omega_g/2)

    exp_xop = exp(im*eta_r*xop)
    add_unitary!(ham, (1, 2), ("e<-r", exp_xop), coeff=omega_r/2)
    add_unitary!(ham, (1, 2), ("r<-e", exp_xop'), coeff=omega_r/2)
    
    dtheta = 0.01
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_g*theta*xop)
        add_dissipation!(ham, (1, 2), ("g<-e", jump), coeff=gamma_g/2)
    end
    
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_r*theta*xop)
        add_dissipation!(ham, (1, 2), ("r<-e", jump), coeff=gamma_r/2)
    end
    
    observer = EDObservers(ham)
    add!(observer, (1,), ("e<-e",), name="e")
    add!(observer, (1,), ("g<-g",), name="g")
    add!(observer, (1,), ("r<-r",), name="r")
    add!(observer, (2,), ("n",), name="n")
    
    L = size(state, 1)

    ham = immrep(ham)
    ham = matrix(ham)
    state = reshape(state, L*L)
    
    observer = immrep(observer)
    result = measure(observer, reshape(state, L, L))
    obs = Observables(result)
    mt = t/100
    n = round(Int, t/mt)
    for tj in 1:n
        state = evolve(ham, mt, state, ishermitian=false)
        tmp = measure(observer, reshape(state, L, L))
        append!(obs, tmp)
    end
    
    return obs
end

function eit_cooling2(d, Delta1,Delta2, nu, eta_g, omega_g, eta_r, omega_r, gamma_g, gamma_r, state, t)
    ps = nlevel(["g", "r", "e"])
    pb = boson(d=d)
    
    xop = pb["a"] + pb["adag"]
    
    ham = Lindbladian([ps, pb])
    add_unitary!(ham, (1,), ("e<-e",), coeff=-Delta1)
    add_unitary!(ham, (1,), ("r<-r",), coeff=-(Delta1-Delta2))
    add_unitary!(ham, (2,), ("n",), coeff=nu)
    
    exp_xop = exp(im*eta_g*xop)
    add_unitary!(ham, (1, 2), ("e<-g", exp_xop), coeff=omega_g/2)
    add_unitary!(ham, (1, 2), ("g<-e", exp_xop'), coeff=omega_g/2)

    exp_xop = exp(im*eta_r*xop)
    add_unitary!(ham, (1, 2), ("e<-r", exp_xop), coeff=omega_r/2)
    add_unitary!(ham, (1, 2), ("r<-e", exp_xop'), coeff=omega_r/2)
    
    dtheta = 0.01
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_g*theta*xop)
        add_dissipation!(ham, (1, 2), ("g<-e", jump), coeff=gamma_g/2)
    end
    
    for theta in -1:dtheta:1
        c = dtheta * 0.75 * (1 + theta^2)
        jump = sqrt(c / 2) * exp(im*eta_r*theta*xop)
        add_dissipation!(ham, (1, 2), ("r<-e", jump), coeff=gamma_r/2)
    end
    
    observer = EDObservers(ham)
    add!(observer, (1,), ("e<-e",), name="e")
    add!(observer, (1,), ("g<-g",), name="g")
    add!(observer, (1,), ("r<-r",), name="r")
    add!(observer, (2,), ("n",), name="n")
    
    L = size(state, 1)

    ham = immrep(ham)
    ham = matrix(ham)
    state = reshape(state, L*L)
    
    observer = immrep(observer)
    result = measure(observer, reshape(state, L, L))
    obs = Observables(result)
    mt = t/100
    n = round(Int, t/mt)
    for tj in 1:n
        state = evolve(ham, mt, state, ishermitian=false)
        tmp = measure(observer, reshape(state, L, L))
        append!(obs, tmp)
    end
    
    return obs
end

eit_cooling2 (generic function with 1 method)

In [16]:
ttot1=300
function plotme()
    ym=[]
    yd=[]
    nn=a₊(dim)*a₋(dim)
    num=100
    state=instate
    x=[real(tr(kron([0 0 0;0 0 0;0 0 1],eye(dim))*vec2dm(state)))]
    ym=push!(ym,ptrace([dim,3],vec2dm(state),[2]))
    z=[real(tr(nn*ptrace([dim,3],vec2dm(state),[2])))]
    yd=push!(yd,0.0)
    for i in 1:num
        state= exp(ttot1/num*eit_Lindbladian_vec(3.6103223565556357,21.05702368447422,111.9645147569779))*state
        append!(x,real(tr(kron([0 0 0;0 0 0;0 0 1],eye(dim))*vec2dm(state))))    
        pp=ptrace([10,3],vec2dm(state),[2])
        push!(ym,pp)
        append!(z,real(tr(nn*pp)))
        append!(yd,norm(thermal_state(dim,z[length(z)])-ym[length(ym)]))
        print(i," ")
    end
    return z,yd
    
#     plot(yticks=0:0.2:1,xlabel="t/ν^-1",ylabel="n")
#     plot!(0:ttot1/100:ttot1,z);plot!(0:ttot1/100:ttot1,yd)
#     plot!(0:ttot1/100:ttot1,eit_cooling2(dim, 58.908208592426824,58.682468257711506, ν, η1, 5.281092614609339, η2, 16.28498178068964, γ1, γ2, kron(thermal_state(dim,1),[1.0 0 0;0 0 0;0 0 0]), ttot1)["n[2]"])
#     plot!(0:ttot1/100:ttot1,eit_cooling(dim, 111.9645147569779, ν, η1, 3.6103223565556357, η2, 21.05702368447422, γ1, γ2, kron(thermal_state(dim,1),[1.0 0 0;0 0 0;0 0 0]), ttot1)["n[2]"])

end

plt1=plotme()#用来做对比

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 

([0.9902248289345064, 0.9766337912079649, 0.9377797724233087, 0.8896015826228996, 0.8317919945826714, 0.7776827724236288, 0.7244270891279434, 0.6771984912785685, 0.6341150178833534, 0.5951775924052991  …  0.00811074804682977, 0.007917272881815415, 0.007735448320265802, 0.007566064843547524, 0.007407024905322365, 0.007258763620071865, 0.007119712396287363, 0.006989971003927297, 0.0068684384639759654, 0.006754931315037862], Any[0.0, 0.011004013087380813, 0.017628130246036038, 0.025974879589934618, 0.030274301832289637, 0.03750063697111907, 0.041947173398730636, 0.048928650767809695, 0.054606459198200064, 0.061518235966296776  …  0.003414373093411835, 0.0032941649641465734, 0.003176487202693092, 0.0030735656887039497, 0.0029711925513190678, 0.002883098714011409, 0.0027943648491064567, 0.0027188638010482444, 0.0026422331783778124, 0.0025773366497287894])

In [18]:
plot(yticks=0:0.2:1,xlabel="t/ν^-1",ylabel="n")
plot!(0:ttot1/100:ttot1,plt1[1],label="nbar");plot!(0:ttot1/100:ttot1,plt1[2],label="distance")

<!DOCTYPE html>
 
 
 Plots.jl